<a href="https://colab.research.google.com/github/UpwardTrajectory/auto-rapper/blob/master/auto_rapper_w_gpu_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto-Rapper
##  Train a GPT-2 Text-Generating Model w/ GPU For Free 
## WARNING -- Lyrics are unpredictable and may be explicit or disturbing -- WARNING

Based on the Colab by  [Max Woolf](http://minimaxir.com)

*Last updated: June 21th, 2019*

Retrain an advanced text generating neural network on any text dataset **for free on a GPU using Collaboratory** using `gpt-2-simple`!

For more about `gpt-2-simple`, you can visit [this GitHub repository](https://github.com/minimaxir/gpt-2-simple).


To get started:

1. Copy this notebook to your Google Drive to keep it and save your changes. (File -> Save a Copy in Drive)
2. Make sure you're running the notebook in Google Chrome.
3. Run the cells below:


In [0]:
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

## GPU

Colaboratory now uses an Nvidia T4 GPU, which is slightly faster than the old Nvidia K80 GPU for training GPT-2, and has more memory allowing you to train the larger GPT-2 models and generate more text.

You can verify which GPU is active by running the cell below.

In [0]:
!nvidia-smi

Wed Jun 19 23:20:25 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are two sizes of GPT-2:

* `117M` (default): the "small" model, 500MB on disk.
* `345M`: the "medium" model, 1.5GB on disk.

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [0]:
gpt2.download_gpt2(model_name="345M")

Fetching checkpoint: 1.05Mit [00:00, 270Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 114Mit/s]                                                    
Fetching hparams.json: 1.05Mit [00:00, 263Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:10, 132Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 263Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 115Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 203Mit/s]                                                       


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [2]:
gpt2.mount_gdrive()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

## Caution:
For this cell to work, you must do the following steps:  

 1) create the `rapper.txt` file containing the corpus of lyrics using `lyrics_dl.ipynb` on our github.  
 2) copy those texts into the root folder of your google drive.  
 3) update the `to_do` list with the correct file name(s).  
 4) restart the runtime environment for each new .txt file you want to fine-tune, and update the `to_do` list accordingly.
 

In [0]:
all_rappers = [
    'eminem.txt', 'beastie_boys.txt', 'aesop_rock.txt', 'common.txt', 
    'jayz.txt', 'mf_doom.txt', 'missy_elliott.txt', 'nas.txt', 'rza.txt'
]

to_do = ['kendrick_lamar.txt']  # no .txt file exists for this yet

file_name = to_do[0]

gpt2.copy_file_from_gdrive(file_name)
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='345M',
              steps=1000,
              learning_rate=1e-5,
              restore_from='fresh',
              run_name=file_name[:-4],
              print_every=25,
              sample_every=200,
              save_every=500
              )

gpt2.copy_checkpoint_to_gdrive(run_name=file_name[:-4])

W0618 21:23:16.257503 139864579749760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:90: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0618 21:23:16.259669 139864579749760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:100: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0618 21:23:16.579629 139864579749760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/gpt_2.py:164: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0618 21:23:16.584361 139864579749760 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/gpt_2_simple/src/model.py:148: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0618 21:23:28.652480 139864579749760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/gpt_2_sim

Loading checkpoint models/345M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


dataset has 135029 tokens
Training...
[10 | 26.10] loss=2.87 avg=2.87
[20 | 41.20] loss=2.93 avg=2.90
[30 | 56.44] loss=3.40 avg=3.07
[40 | 71.80] loss=2.58 avg=2.94
[50 | 87.28] loss=2.78 avg=2.91
[60 | 102.90] loss=2.90 avg=2.91
[70 | 118.60] loss=2.69 avg=2.88
[80 | 134.43] loss=3.03 avg=2.90
[90 | 150.34] loss=3.24 avg=2.94
[100 | 166.35] loss=2.60 avg=2.90
[110 | 182.43] loss=2.93 avg=2.90
[120 | 198.52] loss=2.73 avg=2.89
[130 | 214.59] loss=2.73 avg=2.87
[140 | 230.62] loss=2.87 avg=2.87
[150 | 246.67] loss=2.76 avg=2.87
[160 | 262.69] loss=2.49 avg=2.84
[170 | 278.75] loss=2.86 avg=2.84
[180 | 294.82] loss=2.49 avg=2.82
[190 | 310.87] loss=3.22 avg=2.84
[200 | 326.96] loss=2.85 avg=2.84
======== SAMPLE 1 ========
 like your hair is dry and it's cold
 You don't even eat it or drink it like its water
 You can't sleep on the bus anymore cause you got a headache now it's time to go
 Let me show you my new haircut I cut this shit down
 And you know what I'm up to now you can find me

W0618 21:52:43.313658 139864579749760 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to delete files with this prefix.


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [0]:
gpt2.copy_checkpoint_to_gdrive(run_name=file_name[:-4])

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [0]:
artists = [x[:-4] for x in all_rappers]

for artist in artists:
  gpt2.copy_checkpoint_from_gdrive(run_name=artist)

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=artist)

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model. 

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

---

We will combine some logic into one cell, so we only need to `shift + enter` once, regardless of whether we restarted the runtime enviroment or not.  

### The available rappers are: 

```'eminem',
 'beastie_boys',
 'aesop_rock',
 'common',
 'jayz',
 'mf_doom',
 'missy_elliott',
 'nas',
 'rza'```
 
 ### NOTE: You must restart if you change artists, but not if you want another verse from the same artist.

In [4]:
first_line = "Babys are cute, they bring me joy"
artist = 'eminem'

if artist == 'eminem':
  artist_run = 'run1'
else:
  artist_run = artist

try:
  output = gpt2.generate(
       sess,
       run_name=artist_run,
       length=500,
       temperature=0.80,
       prefix=first_line,
       nsamples=1,
       batch_size=1
       )

  print(output)
  
except:
  import gpt_2_simple as gpt2
  #gpt2.copy_checkpoint_from_gdrive(run_name=artist)
  sess = gpt2.start_tf_sess()

  gpt2.load_gpt2(sess, run_name=artist_run)

  output = gpt2.generate(
       sess,
       run_name=artist_run,
       length=500,
       temperature=0.80,
       prefix=first_line,
       nsamples=1,
       batch_size=1
       )

  print(output)

Babys are cute, they bring me joy 
 When they see me with a male companion 
 They run, they jump, they act all wild 
 But they know they're in the wrong neighborhood 
 They see me throw punches, I don't mean to push the point 
 But that Mr. K-Fed is a man, the real beef is with the motherfuckers 
 That's why you see 'em clapping hands, and they're loud 
 But you hear 'em saying, "Go, David, go!" 
 So go, David, go! 
 But don't go in there just yet 
  
 [Chorus: Royce da 5'9"] 
 On the one hand, I got meat on my plate 
 But every now and then I get a craving 
 To be on the other hand, you see 
 I'ma make you eat my foil 
 But first, I'd like to toast 
  
 [Verse 5: Royce da 5'9"] 
 I ain't known no one like you 
 I'd make the world a better place 
 If you only knew the truth 
 I'ma make you eat my foil 
 But first, I'd like to toast 
  
 [Chorus: Royce da 5'9"] 
 On the one hand, I got meat on my plate 
 But every now and then I get a craving 
 To be on the other hand, you see 
 I'ma ma

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp and then download it.

You can rerun the cell as many times as you want for even more generated texts!

In [0]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

files.download(gen_file)

# Etcetera

If the notebook has errors (e.g. GPU Sync Fail or out-of-memory/OOM), force-kill the Colaboratory virtual machine and restart it with the command below:

In [0]:
!kill -9 -1

# LICENSE

MIT License

Copyright (c) 2019 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.